## Bem-vindo! 

##### Esse projeto visa simular um corpo em queda livre, sob a ação da resistência do ar, ou não, com o intuito de analisar os princípios de conservação e dissipação de energia (proposta de projeto 4)

**Feito por:** Arthur de Carvalho, Marcel Figueredo e Paulo de Pinho

**Bibliotecas usadas:** Math, Numpy e Matplotlib 

**Constante usada para aceleração da gravidade:**  9,8 m/s² 


### Instruções de Uso:

1.  **Execute as células em ordem:** Comece pela primeira célula de código (importação de bibliotecas) e execute-as sequencialmente.
    * *Atalho: Pressione `Shift + Enter` para executar uma célula e ir para a próxima.*
2.  **Execute tudo de uma vez (Opcional):**
    * Se preferir, você pode ir ao menu superior e selecionar `Run` > `Run All Cells` (Executar Tudo).
    * Isso irá limpar todas as saídas anteriores, pedir os dados de entrada e executar o notebook do início ao fim.
3.  **Insira os dados de entrada:** Ao executar a segunda célula de código (`declarando os valores iniciais`), você precisará digitar três valores:
    * **Massa (Kg):** A massa do objeto (ex: `80`).
    * **Altura (m):** A altura inicial da queda (ex: `100`).
    * **Coeficiente de Arrasto (k):** Este é o valor mais importante para a análise.
    * ***(Nota sobre decimais):** Você pode inserir números com casas decimais usando tanto **vírgula** (ex: `0,5`) quanto **ponto** (ex: `0.5`).*
4.  **Entendendo o Coeficiente `k`:**
    * **`k = 0`**: Simula uma **queda livre ideal** (no vácuo). Você verá que a energia total se conserva.
    * **`k > 0`**: Simula uma **queda com resistência do ar** (ex: `0.2` ou `0.7`). Você verá a energia total diminuir e um gráfico de energia dissipada aparecer.
5.  **Analise os resultados:** As últimas células irão calcular e exibir os gráficos de energia (Potencial, Cinética e Dissipada) com base nos valores que você forneceu.
6.  **Aviso sobre Overflow (Erro de Estouro):**    
    * O código possui uma trava de segurança (`OverflowError`) para cálculos com resistência do ar (k > 0).    
    * Se os valores de **altura (`h`)** e **coeficiente (`k`)** forem **muito altos** ao mesmo tempo em que a **massa (`m`)** for **muito baixa**, a simulação será interrompida para evitar um erro de estouro matemático. 
    * Se isso acontecer, execute o notebook novamente (`Run` > `Run All Cells`) com valores mais equilibrados.

In [ ]:
# importando bibliotecas:
import math
import numpy as np
import matplotlib.pyplot as plt

# funções:
def ler_valor(string):
    return float(input(string).replace(',', '.')) # função replace() para permitir que o usuário insira um número com casas decimais com separação por vírgula

In [ ]:
# lendo e declarando os valores iniciais:
g = 9.8 # declarando constante de aceleração da gravidade

m_objeto = ler_valor("Insira a massa do objeto (Kg): ") # lendo valor da massa do objeto
while (m_objeto <= 0): # verificando se há erros para o valor da massa
    m_objeto = ler_valor("Valor inválido para a massa do objeto (Kg). Insira um novo valor maior que zero: ") # lendo novamente o valor em caso de erro

h_objeto = ler_valor("Insira a distância entre o objeto e o chão (m): ") # lendo valor da distância do objeto em relação ao chão
while (h_objeto <= 0): # verificando se há erros para o valor da altura
    h_objeto = ler_valor("Valor inválido para a distância do objeto ao chão (m). Insira um novo valor maior que zero: ") # lendo novamente o valor em caso de erro

k = ler_valor("Insira o coeficiente de arrasto (0 para ignorar a resistência do ar): ") # lendo coeficiente de arrasto
while (k < 0): # verificando se há erros para o coeficiente de arrasto
    k = ler_valor("Valor inválido para o coeficiente de arrasto. Insira um novo valor maior ou igual a zero") # lendo novamente o valor em caso de erro

# verificando se há um erro de desproporção entre as medidas:
if (k * h_objeto) / m_objeto >= 700: # verificando erros de desproporção extrema entre os valores
    raise OverflowError("ERRO: A combinação de k, altura e massa é muito extrema e causa um Overflow. Rode o código novamente e tente novos valores.") # exibindo mensagem de erro

In [ ]:
# implementando resistência do ar:
if (k != 0): # Considerando resistência do ar no cálculo da altura durante a queda:
    v_terminal = math.sqrt(m_objeto * g / k) # velocidade terminal
    
    termo_exp = math.exp((g * h_objeto) / (v_terminal**2)) # uma variável auxiliar para calcular o tempo real de queda, considerando a resistência do ar 
    t_final = (v_terminal / g) * math.acosh(termo_exp) # calculando o tempo final de queda, considerando a resistência do ar
    t_queda = np.linspace(0, t_final, num=1000) # tempo de queda em diferentes instantes do movimento
    
    v = v_terminal * np.tanh((g / v_terminal) * t_queda) # velocidade do objeto ao longo do movimento
    
    h_t = h_objeto - (m_objeto / k) * np.log(np.cosh((k / m_objeto) * t_queda * v_terminal)) # altura do objeto em função do tempo
    h_t = np.maximum(h_t, 0.0) # limitando a altura a zero (para que o objeto não atravesse o chão após o impacto)
    
else: # Desconsiderando resistência do ar no cálculo da altura durante a queda.
    t_final = math.sqrt(2 * h_objeto / g) # tempo de queda final
    t_queda = np.linspace(0, t_final, num=1000) # tempo de queda em diferentes instantes

    h_t = h_objeto - 0.5 * g * t_queda**2 # computando altura como função do tempo sob a queda livre
    h_t = np.maximum(h_t, 0.0) # limitando a altura a zero (para que o objeto não atravesse o chão após o impacto)

    v = g * t_queda # velocidade do objeto ao longo do movimento

# calculando energia cinética e potencial:
energia_cinetica = 0.5 * m_objeto * v ** 2 # cálculo da energia cinética nos diversos instantes de tempo
energia_potencial = m_objeto * g * h_t # energia potencial gravitacional em relação ao tempo

In [ ]:
# plotando energia potencial gravitacional em função do tempo de queda:
fig, ax = plt.subplots()
ax.plot(t_queda, energia_potencial)
ax.set_title("Energia Potencial Gravitacional x Tempo de Queda")
ax.set_xlabel("Tempo de Queda (s)")
ax.set_ylabel("Energia Potencial Gravitacional (J)")
plt.grid(True)
plt.show()

# plotando energia cinética em função do tempo de queda:
fig, ax = plt.subplots()
ax.plot(t_queda, energia_cinetica)
ax.set_title("Energia Cinética x Tempo de Queda")
ax.set_xlabel("Tempo de Queda (s)")
ax.set_ylabel("Energia Cinética (J)")
plt.grid(True)
plt.show()

# verificando se há dissipação de energia
energia_total = energia_potencial + energia_cinetica # calculando energia total do sistema, somando energia potencial e cinética
energia_total_constante = np.allclose(energia_total, energia_total[0]) # verificando se a energia total do sistema é constante

# estrutura condicional para informar se a energia total do sistema é constante ou não. Se não for, também plota o gráfico da energia dissipada
if (energia_total_constante):
    print("A energia total do sistema é constante ao longo do tempo, conforme esperado pela conservação de energia.")
else:
    print("A energia total do sistema não é constante, logo há alguma perda de energia (devido à resistência do ar).")
    
    # plotando gráfico de dissipação da energia:
    energia_dissipada = m_objeto * g * h_objeto - energia_total # calculando a energia dissipada pela resistência do ar
    fig, ax = plt.subplots()
    ax.plot(t_queda, energia_dissipada)
    ax.set_title("Energia Dissipada x Tempo de Queda")
    ax.set_xlabel("Tempo de Queda (s)")
    ax.set_ylabel("Energia Dissipada (J)")
    plt.grid(True)
    plt.show()